In [1]:
import os
import ast
from pydantic import create_model
import inspect, json
from inspect import Parameter
from configparser import ConfigParser
from fastcore.utils import nested_idx
from openai import OpenAI

In [2]:
# !pip install fastcore
# !pip install openai

## Helpers

In [3]:
config=ConfigParser()
config.read('conf/conf.ini')
os.environ["OPENAI_API_KEY"] = config['openai']['apikey']

client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')

In [4]:
def askgpt(user, system=None, model="gpt-3.5-turbo", **kwargs):
    msgs = []
    if system: msgs.append({"role": "system", "content": system})
    msgs.append({"role": "user", "content": user})
    return client.chat.completions.create(model=model, messages=msgs, **kwargs)

In [5]:
def response(compl): print(nested_idx(compl, 'choices', 0, 'message', 'content'))

## Define function and schema

In [6]:
def sums(a:int, b:int=1):
    "Adds a + b"
    return a + b

In [7]:
def schema(f):
    kw = {n:(o.annotation, ... if o.default==Parameter.empty else o.default)
          for n,o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [8]:
schema(sums)

{'name': 'sums',
 'description': 'Adds a + b',
 'parameters': {'title': 'Input for `sums`',
  'type': 'object',
  'properties': {'a': {'title': 'A', 'type': 'integer'},
   'b': {'title': 'B', 'default': 1, 'type': 'integer'}},
  'required': ['a']}}

## Show query to LLM and output how it as identified function adn arguments to call

In [9]:
c = askgpt("Use the `sum` function to solve this: What is 6+3?",
           system = "You must use the `sum` function instead of adding yourself.",
           functions=[schema(sums)])

In [10]:
c.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "a": 6,\n  "b": 3\n}', name='sums'), tool_calls=None)

In [11]:
print(c.choices[0].message.content)

None


In [12]:
m = c.choices[0].message

In [13]:
k = m.function_call.arguments
print(k)

{
  "a": 6,
  "b": 3
}


## Define ok funcs, not to have random funcs called by LLM

In [14]:
funcs_ok = {'sums', 'python'}

In [15]:
def call_func(c):
    fc = c.choices[0].message.function_call
    if fc.name not in funcs_ok: return print(f'Not allowed: {fc.name}')
    f = globals()[fc.name]
    return f(**json.loads(fc.arguments))

In [16]:
call_func(c)

9

In [17]:
def run(code):
    tree = ast.parse(code)
    last_node = tree.body[-1] if tree.body else None
    
    # If the last node is an expression, modify the AST to capture the result
    if isinstance(last_node, ast.Expr):
        tgts = [ast.Name(id='_result', ctx=ast.Store())]
        assign = ast.Assign(targets=tgts, value=last_node.value)
        tree.body[-1] = ast.fix_missing_locations(assign)

    ns = {}
    exec(compile(tree, filename='<ast>', mode='exec'), ns)
    return ns.get('_result', None)

In [18]:
run("""
a=1
b=2
a+b
""")

3

In [24]:
def python(code:str):
    "Return result of executing `code` using python. If execution not permitted, returns `#FAIL#`"
    go = input(f'Proceed with execution?\n```\n{code}\n```\n')
    if go.lower()!='y': return '#FAIL#'
    return run(code)

In [25]:
c = askgpt("What is 12 factorial?",
           system = "Use python for any required computations.",
           functions=[schema(python)])

In [26]:
call_func(c)

Proceed with execution?
```
import math
math.factorial(12)
```
 y


479001600

In [27]:
c = client.chat.completions.create(
    model="gpt-3.5-turbo",
    functions=[schema(python)],
    messages=[{"role": "user", "content": "What is 12 factorial?"},
              {"role": "function", "name": "python", "content": "479001600"}])

In [28]:
c.choices[0].message.content

'12 factorial, denoted as 12!, is the product of all positive integers from 1 to 12. \n\n12! = 12 × 11 × 10 × 9 × 8 × 7 × 6 × 5 × 4 × 3 × 2 × 1 = 479,001,600.'

## Dummy example with my function

In [29]:
funcs_ok = {'find_trends', 'python'}

In [30]:
def find_trends(first_period_start:str, first_period_end:str, 
                second_period_start:str, second_period_end:str,
                channel:str):
    """Finds trends by comparing two time periods: first_period_start-first_period_end vs second_period_start-second_period_end in a specified channel
    first_period_start - start of the first period, in format dd.mm.yyyy
    first_period_end - end of the first period, in format dd.mm.yyyy
    second_period_start - start of the second period, in format dd.mm.yyyy
    second_period_end - end of the second period, in format dd.mm.yyyy
    channel - name of the channel where to look for trends"""
    if channel=='call':
        return """topic 'invoice' rose 12%, 
              topic 'payment' decreased 2%""" 
    else:
        return "something rose and something decreased"

In [31]:
c = askgpt("Find trends between 01.01.2023-05.01.2023 vs 01.02.2023-05.02.2023 in channel 'call'",
           #system = "You must use the `sum` function instead of adding yourself.",
           functions=[schema(find_trends)])

In [32]:
c

ChatCompletion(id='chatcmpl-8V0qNLfQwEeD5TK8YnVaZcZb2nZEl', choices=[Choice(finish_reason='function_call', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "first_period_start": "01.01.2023",\n  "first_period_end": "05.01.2023",\n  "second_period_start": "01.02.2023",\n  "second_period_end": "05.02.2023",\n  "channel": "call"\n}', name='find_trends'), tool_calls=None))], created=1702402103, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=72, prompt_tokens=212, total_tokens=284))

In [33]:
c.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "first_period_start": "01.01.2023",\n  "first_period_end": "05.01.2023",\n  "second_period_start": "01.02.2023",\n  "second_period_end": "05.02.2023",\n  "channel": "call"\n}', name='find_trends'), tool_calls=None)

In [34]:
c.choices[0].message.function_call

FunctionCall(arguments='{\n  "first_period_start": "01.01.2023",\n  "first_period_end": "05.01.2023",\n  "second_period_start": "01.02.2023",\n  "second_period_end": "05.02.2023",\n  "channel": "call"\n}', name='find_trends')

In [35]:
print(call_func(c))

topic 'invoice' rose 12%, 
              topic 'payment' decreased 2%


In [36]:
c = askgpt("What is capital of France",
           #system = "You must use the `sum` function instead of adding yourself.",
           functions=[schema(find_trends)])
c.choices[0].message

ChatCompletionMessage(content='The capital of France is Paris.', role='assistant', function_call=None, tool_calls=None)

In [37]:
def askgpt_func_call(user, system=None, model="gpt-3.5-turbo", **kwargs):
    msgs = []
    if system: msgs.append({"role": "system", "content": system})
    msgs.append({"role": "user", "content": user})
    c=client.chat.completions.create(model=model, messages=msgs, **kwargs)
    
    if c.choices[0].message.function_call is None:
        return c.choices[0].message.content
    func_call_out=call_func(c)
    
    msgs.append({"role": "function", "name": "python", "content": func_call_out})
    c2=client.chat.completions.create(model=model, messages=msgs)
    return c2.choices[0].message.content

In [38]:
resp=askgpt_func_call("Find trends between 01.01.2023-05.01.2023 vs 01.02.2023-05.02.2023 in channel 'call'",
                functions=[schema(find_trends)])

In [39]:
print(resp)

During the period from 01.01.2023-05.01.2023 to 01.02.2023-05.02.2023, there were some notable trends in the 'call' channel related to the topics of 'invoice' and 'payment'. 

The topic of 'invoice' experienced a significant rise of 12% during this period. This suggests that there was a higher volume of calls related to invoices in the later timeframe compared to the earlier timeframe.

On the other hand, the topic of 'payment' witnessed a decrease of 2% in the later timeframe. This indicates that there were slightly fewer calls related to payments during the second period compared to the first period.

Overall, these trends suggest that there was an increased focus on invoices and a slight decline in calls related to payments in the 'call' channel between 01.01.2023-05.01.2023 and 01.02.2023-05.02.2023.


In [40]:
client